In [3]:
import nibabel as nib
import matplotlib.pyplot as plt

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
import numpy as np
import torchvision.models as models

In [6]:
# 自定义数据集类
class SpineWeb15(Dataset):
    def __init__(self, data_path, transform=None):
        self.data_path = data_path
        self.transform = transform
        
    def __len__(self):
        return len(self.data_path)
    
    def __getitem__(self, index):
        metadata = np.load(self.data_path+str(index)+".npy")
        image = metadata[0]
        label = metadata[1]

        image_data = torch.from_numpy(image).float()
        label_data = torch.from_numpy(label).float()
        
        if self.transform:
            image_data = self.transform(image_data)
        return image_data, label_data



In [7]:
# VGG Net structure

class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        # 加载预训练的VGG模型作为基础模型
        self.vgg = models.vgg16(pretrained=True)
        # 更改第一层的图像输入维度为1，为灰度图像
        self.vgg.features[0] = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        # 替换最后一层全连接层，根据需要调整输出通道数
        self.vgg.classifier[6] = nn.Linear(4096, 128*128)  # 输出通道数为128*128

    def forward(self, x):
        x = self.vgg.features(x)
        x = self.vgg.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.vgg.classifier(x)
        x = x.view(-1, 1, 128, 128)
        return x


In [6]:
# -----------------------------
# VGG16 Net All Set
# -----------------------------

# empty the cache for model and training process
torch.cuda.empty_cache()

# set the hyper parameters and the paths 设置超参数和路径
data_path = "./dataset/Spineweb_dataset15/processed/"

#hyper parameters set
batch_size = 16
num_epochs = 50
learning_rate = 0.001
confidence = 0.5
in_channels = 1  # according to demand to adjust the number of channels 根据实际情况修改通道数
out_channels = 1  # according to demand to adjust the number of channels 根据实际情况修改通道数

# 创建数据集和数据加载器
transform = transforms.Compose([
    transforms.ToPILImage(),  # 转换为 PIL 图像对象
    transforms.Resize((128, 128)),  # 调整大小为 128x128
    transforms.ToTensor()  # 转换为张量
])
dataset = SpineWeb15(data_path=data_path,transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 创建模型和优化器
model = VGG16()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCEWithLogitsLoss()  # 二分类任务可以使用BCEWithLogitsLoss

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# check the parameters of the model could be trained 检查模型的参数是否设置为可训练
# for name, param in model.named_parameters():
#     print(f"Parameter: {name}, Requires Grad: {param.requires_grad}")

best_loss = 100.0
best_IoU = 0.0
best_Dice = 0.0

# 训练模型
for epoch in range(num_epochs):
    torch.cuda.empty_cache()
    model.train()
    train_loss = 0
    total_iou = 0
    total_dice = 0

    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)
    
        optimizer.zero_grad()

        outputs = model(images)
        # squeeze the dimension that pytorch add for channels
        outputs = outputs.squeeze(1)     # tensor size ([16,128,128])
        # process the result, bigger than 0.5 is reliable
        predicted = (outputs > confidence).float()     # tensor size ([16,128,128])

        intersection = torch.logical_and(predicted, labels).sum((1, 2))
        union = torch.logical_or(predicted, labels).sum((1, 2))
        iou = (intersection / (union + 1e-7)).mean()  # 平均每个样本的IoU
        dice = (2 * intersection / (predicted.sum((1, 2)) + labels.sum((1, 2)) + 1e-7)).mean()  # 平均每个样本的Dice Coefficient

        loss = criterion(outputs, labels)

        train_loss += loss.item()
        total_iou += iou.item()
        total_dice += dice.item()

        loss.backward()
        optimizer.step()

    avg_loss = train_loss / len(dataloader)
    avg_iou = total_iou / len(dataloader)
    avg_dice = total_dice / len(dataloader)
    
    # get the best IoU and Dice
    best_loss = best_loss if best_loss < avg_loss else avg_loss
    best_IoU = best_IoU if best_IoU > avg_iou else avg_iou
    best_Dice = best_Dice if best_Dice > avg_dice else avg_dice

    print(f"Epoch [{epoch+1}/{num_epochs}], Avg Loss: {avg_loss:.4f}, Avg IoU: {avg_iou:.4f}, Avg Dice: {avg_dice:.4f}")
print("-----------------------------------------")
print(f"Best Loss: {best_loss:.4f}, Best IoU: {best_IoU:.4f}, Best Dice: {best_Dice:.4f}")
# empty the cache for model and training process
torch.cuda.empty_cache()

Epoch [1/50], Avg Loss: 0.5607, Avg IoU: 0.0609, Avg Dice: 0.0962
Epoch [2/50], Avg Loss: 0.1797, Avg IoU: 0.1335, Avg Dice: 0.2077
Epoch [3/50], Avg Loss: 0.0785, Avg IoU: 0.0998, Avg Dice: 0.1631
Epoch [4/50], Avg Loss: 0.0685, Avg IoU: 0.0457, Avg Dice: 0.0830
Epoch [5/50], Avg Loss: 0.0691, Avg IoU: 0.0205, Avg Dice: 0.0387
Epoch [6/50], Avg Loss: 0.0673, Avg IoU: 0.1005, Avg Dice: 0.1671
Epoch [7/50], Avg Loss: 0.0622, Avg IoU: 0.1321, Avg Dice: 0.2038
Epoch [8/50], Avg Loss: 0.0721, Avg IoU: 0.1294, Avg Dice: 0.1956
Epoch [9/50], Avg Loss: 0.0590, Avg IoU: 0.0652, Avg Dice: 0.1122
Epoch [10/50], Avg Loss: 0.0678, Avg IoU: 0.0282, Avg Dice: 0.0505
Epoch [11/50], Avg Loss: 0.0770, Avg IoU: 0.0144, Avg Dice: 0.0263
Epoch [12/50], Avg Loss: 0.0629, Avg IoU: 0.0390, Avg Dice: 0.0691
Epoch [13/50], Avg Loss: 0.0614, Avg IoU: 0.1510, Avg Dice: 0.2265
Epoch [14/50], Avg Loss: 0.0612, Avg IoU: 0.0560, Avg Dice: 0.0920
Epoch [15/50], Avg Loss: 0.0627, Avg IoU: 0.0256, Avg Dice: 0.0478
Epoc

In [8]:
# -----------------------------
# VGG16 Train and Test Set
# -----------------------------

# empty the cache for model and training process
torch.cuda.empty_cache()

# set the hyper parameters and the paths 设置超参数和路径
data_path = "./dataset/Spineweb_dataset15/train/"
test_path = "./dataset/Spineweb_dataset15/test/"

#hyper parameters set
batch_size = 32
num_epochs = 50
learning_rate = 0.0001
confidence = 0.5
in_channels = 1  # according to demand to adjust the number of channels 根据实际情况修改通道数
out_channels = 1  # according to demand to adjust the number of channels 根据实际情况修改通道数

# 创建数据集和数据加载器
transform = transforms.Compose([
    transforms.ToPILImage(),  # 转换为 PIL 图像对象
    transforms.Resize((128, 128)),  # 调整大小为 128x128
    transforms.ToTensor()  # 转换为张量
])
dataset = SpineWeb15(data_path=data_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
testset = SpineWeb15(data_path=test_path, transform=transform)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=True)

# 创建模型和优化器
model = VGG16()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCEWithLogitsLoss()  # 二分类任务可以使用BCEWithLogitsLoss

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# check the parameters of the model could be trained 检查模型的参数是否设置为可训练
# for name, param in model.named_parameters():
#     print(f"Parameter: {name}, Requires Grad: {param.requires_grad}")

best_loss_train = 100.0
best_IoU_train = 0.0
best_Dice_train = 0.0

best_IoU_test = 0.0
best_Dice_test = 0.0

# 训练模型
for epoch in range(num_epochs):
    # torch.cuda.empty_cache()
    model.train()
    train_loss = 0
    total_iou = 0
    total_dice = 0
    total_iou_test = 0
    total_dice_test = 0

    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)
    
        optimizer.zero_grad()

        outputs = model(images)
        # squeeze the dimension that pytorch add for channels
        outputs = outputs.squeeze(1)     # tensor size ([16,128,128])
        # process the result, bigger than 0.5 is reliable
        predicted = (outputs > confidence).float()     # tensor size ([16,128,128])

        intersection = torch.logical_and(predicted, labels).sum((1, 2))
        union = torch.logical_or(predicted, labels).sum((1, 2))
        iou = (intersection / (union + 1e-7)).mean()  # 平均每个样本的IoU
        dice = (2 * intersection / (predicted.sum((1, 2)) + labels.sum((1, 2)) + 1e-7)).mean()  # 平均每个样本的Dice Coefficient

        loss = criterion(outputs, labels)

        train_loss += loss.item()
        total_iou += iou.item()
        total_dice += dice.item()

        loss.backward()
        optimizer.step()

    # train set results
    avg_loss = train_loss / len(dataloader)
    avg_iou = total_iou / len(dataloader)
    avg_dice = total_dice / len(dataloader)

    # test set results
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            # squeeze the dimension that pytorch add for channels
            outputs = outputs.squeeze(1)     # tensor size ([16,128,128])
            # process the result, bigger than 0.5 is reliable
            predicted = (outputs > confidence).float()     # tensor size ([16,128,128])
            
            intersection = torch.logical_and(predicted, targets).sum((1, 2))
            union = torch.logical_or(predicted, targets).sum((1, 2))
            iou = (intersection / (union + 1e-7)).mean()  # 平均每个样本的IoU
            dice = (2 * intersection / (predicted.sum((1, 2)) + targets.sum((1, 2)) + 1e-7)).mean()  # 平均每个样本的Dice Coefficient

            total_iou_test += iou.item()
            total_dice_test += dice.item()

    test_iou = total_iou_test / len(test_loader)
    test_dice = total_dice_test / len(test_loader)

    # get the best IoU and Dice
    best_loss_train = best_loss_train if best_loss_train < avg_loss else avg_loss
    best_IoU_train = best_IoU_train if best_IoU_train > avg_iou else avg_iou
    best_Dice_train = best_Dice_train if best_Dice_train > avg_dice else avg_dice
    best_IoU_test = best_IoU_test if best_IoU_test > test_iou else test_iou
    # best_Dice_test = best_Dice_test if best_Dice_test > test_dice else test_dice

    if(best_Dice_test <= test_dice):
        best_Dice_test = test_dice
        torch.save(model.state_dict(), f"./model/VGG16_checkpoint.pth")

    print(f"Epoch [{epoch+1}/{num_epochs}], Avg Loss: {avg_loss:.4f}, Avg IoU: {avg_iou:.4f}, Avg Dice: {avg_dice:.4f},test IoU: {test_iou:.4f}, test Dice: {test_dice:.4f}")
    
print("-----------------------------------------")
print(f"Best Loss: {best_loss_train:.4f}, Best Train IoU: {best_IoU_train:.4f}, Best Train Dice: {best_Dice_train:.4f}, Best Test IoU: {best_IoU_test:.4f}, Best Test Dice: {best_Dice_test:.4f}")
# empty the cache for model and training process
torch.cuda.empty_cache()

Epoch [1/50], Avg Loss: 0.6960, Avg IoU: 0.0048, Avg Dice: 0.0095,test IoU: 0.0000, test Dice: 0.0000
Epoch [2/50], Avg Loss: 0.6673, Avg IoU: 0.0070, Avg Dice: 0.0137,test IoU: 0.0165, test Dice: 0.0320
Epoch [3/50], Avg Loss: 0.5756, Avg IoU: 0.0303, Avg Dice: 0.0574,test IoU: 0.0603, test Dice: 0.1126
Epoch [4/50], Avg Loss: 0.3667, Avg IoU: 0.0446, Avg Dice: 0.0811,test IoU: 0.0899, test Dice: 0.1626
Epoch [5/50], Avg Loss: 0.1646, Avg IoU: 0.0749, Avg Dice: 0.1289,test IoU: 0.1105, test Dice: 0.1954
Epoch [6/50], Avg Loss: 0.1279, Avg IoU: 0.1552, Avg Dice: 0.2551,test IoU: 0.0809, test Dice: 0.1427
Epoch [7/50], Avg Loss: 0.0699, Avg IoU: 0.0639, Avg Dice: 0.1035,test IoU: 0.0780, test Dice: 0.1382
Epoch [8/50], Avg Loss: 0.0594, Avg IoU: 0.1533, Avg Dice: 0.2417,test IoU: 0.0736, test Dice: 0.1310
Epoch [9/50], Avg Loss: 0.0724, Avg IoU: 0.1247, Avg Dice: 0.2042,test IoU: 0.1052, test Dice: 0.1869
Epoch [10/50], Avg Loss: 0.0565, Avg IoU: 0.1003, Avg Dice: 0.1620,test IoU: 0.119